# Speech + Phonetic AAI Model

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

## Utility Functions

In [2]:
#Function for preprocessing data
def delete_params(params):
    '''
    This function remove JX, WC, TRX, TRY, and MS1,2,3 paramter
    '''
    DEL_PARAMS_LIST = [2,8,15,16,21,22,23]
    return np.delete(params,DEL_PARAMS_LIST , axis=1)

## Load Data 

In [3]:
def prep_data():

    # load data from preprocess pipeline
    dataset = np.load(join('../data/d_dataset_p1/prep_data_13','training_subsets.npz'))
    X_train =dataset['X_train']
    y_train= dataset['y_train']
    z_train= dataset['z_train']
    X_val = dataset['X_val']
    y_val = dataset['y_val']
    z_val = dataset['z_val']
    X_test = dataset['X_test']
    y_test = dataset['y_test']
    z_test = dataset['z_test']
    
    y_train = delete_params(y_train)
    y_val = delete_params(y_val)
    y_test = delete_params(y_test)

    print('Train features and labels %s %s'%(str(X_train.shape),str(y_train.shape)))
    print('Validating features and labels %s %s'%(str(X_val.shape),str(y_val.shape)))
    print('Test features and labels %s %s'%(str(X_test.shape),str(y_test.shape)))

    return X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test

In [4]:
X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test = prep_data()

Train features and labels (98746, 24, 39) (98746, 17)
Validating features and labels (12094, 24, 39) (12094, 17)
Test features and labels (12092, 24, 39) (12092, 17)


In [5]:
print(set(z_test))

{'E', 'Q', 'e', '@', 'M', 'U', '9', '&', '7', 'a', 'E:', 'i', 'o', 'A', 'V', '2', 'O', 'u', 'y'}


In [6]:
print('Train features and labels {} {} {}'.format(str(X_train.shape),str(y_train.shape),str(z_train.shape)))
print('Validating features and labels {} {} {}'.format(str(X_val.shape),str(y_val.shape),str(z_val.shape)))
print('Test features and labels {} {} {}'.format(str(X_test.shape),str(y_test.shape),str(z_test.shape)))

Train features and labels (98746, 24, 39) (98746, 17) (98746,)
Validating features and labels (12094, 24, 39) (12094, 17) (12094,)
Test features and labels (12092, 24, 39) (12092, 17) (12092,)


### One-hot label 

In [7]:
z_train_oe=z_train.reshape(-1,1)
z_val_oe=z_val.reshape(-1,1)
z_test_oe=z_test.reshape(-1,1)

In [8]:
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(z_train_oe)

z_train_oe = enc.transform(z_train_oe).toarray()
z_test_oe = enc.transform(z_test_oe).toarray()
z_val_oe = enc.transform(z_val_oe).toarray()

In [9]:
z_train_oe.shape

(98746, 19)

In [10]:
z_val_oe.shape

(12094, 19)

In [11]:
z_train_oe[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

## Defined Model 

In [12]:
N_OUTPUTS = 17

pLSTM = partial(LSTM,
    kernel_initializer='he_uniform',
    return_sequences=True)

pDense = partial(Dense,
    kernel_initializer='he_normal',
    activation='elu')

def init_bilstm(unit=128, bi_layer_num=3, drop_rate=0.2):

    def bilstm(input_shape_1,input_shape_2):
        
        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        # feature extraction layers
        x = input_x
        for i in range(bi_layer_num-1):
            x = Bidirectional(pLSTM(unit))(x)
            if drop_rate: 
                x = SpatialDropout1D(rate=drop_rate)(x)
        # output layers
        x = Bidirectional(pLSTM(unit*2, return_sequences=False))(x)
        if drop_rate: 
            x = Dropout(rate=drop_rate)(x)
        x = pDense(1024, activation='relu')(x)
        outputs_param = pDense(N_OUTPUTS, activation='linear')(x)
        
        model = keras.Model(inputs=input_x, outputs=outputs_param)
        # model.summary()
        return model
    return bilstm

In [13]:
model = init_bilstm()(24, 39)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 39)]          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 24, 256)           172032    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 24, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 24, 256)           394240    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 24, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1050624   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0     

## Metric Function

In [15]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred), axis = 0) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true, axis = 0)), axis = 0) 
    return K.mean(1 - (SS_res/SS_tot), axis=0)

## Compile Model 

In [16]:
model.compile(optimizer='adam',loss='mse',metrics=[rmse, R2])

## Train Model 

In [17]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 30

In [18]:
early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [19]:
history = model.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val,y_val),
    callbacks=callback_list)

Train on 98746 samples, validate on 12094 samples
Epoch 1/30
98746/98746 [==============================] - 36s 368us/sample - loss: 0.0734 - rmse: 0.2469 - R2: 0.6812 - val_loss: 0.0404 - val_rmse: 0.1809 - val_R2: 0.8270
Epoch 2/30
98746/98746 [==============================] - 21s 216us/sample - loss: 0.0477 - rmse: 0.1954 - R2: 0.7945 - val_loss: 0.0362 - val_rmse: 0.1705 - val_R2: 0.8449
Epoch 3/30
98746/98746 [==============================] - 21s 215us/sample - loss: 0.0415 - rmse: 0.1815 - R2: 0.8215 - val_loss: 0.0343 - val_rmse: 0.1622 - val_R2: 0.8542
Epoch 4/30
98746/98746 [==============================] - 21s 214us/sample - loss: 0.0377 - rmse: 0.1729 - R2: 0.8375 - val_loss: 0.0349 - val_rmse: 0.1619 - val_R2: 0.8497
Epoch 5/30
98746/98746 [==============================] - 21s 215us/sample - loss: 0.0350 - rmse: 0.1669 - R2: 0.8489 - val_loss: 0.0347 - val_rmse: 0.1608 - val_R2: 0.8521
Epoch 6/30
98746/98746 [==============================] - 21s 214us/sample - loss: 0.

In [20]:
model.save('model_baseline.hdf5')

## Evaluation 

In [21]:
model.evaluate(X_test,y_test,verbose=0)

[0.032602244376239105, 0.14959863, 0.8500039]

In [22]:
y_pred = model.predict(X_test)

In [23]:
def compute_rmse(actual,pred, axis=1):
    # Compute RMSE by row (axis=1) result in rmse of each data
    # Compute RMSE by column (axis=0) result in rmse of each label
    return np.sqrt((np.square(actual - pred)).mean(axis=axis))

In [24]:
err = compute_rmse(y_test,y_pred, axis=0)

In [25]:
print(err)

[0.15738225 0.13672984 0.14865203 0.19487172 0.16036555 0.21226606
 0.25415587 0.10173931 0.1306819  0.24195167 0.13813107 0.1652471
 0.17270515 0.26993943 0.18364394 0.15410301 0.15102673]


## Defined Model + Phonetic

In [26]:
N_OUTPUTS = 17

pLSTM = partial(LSTM,
    kernel_initializer='he_uniform',
    return_sequences=True)

pDense = partial(Dense,
    kernel_initializer='he_normal',
    activation='elu')

def init_bilstm_pho(unit=128, bi_layer_num=3, drop_rate=0.2):

    def bilstm(input_shape_1,input_shape_2):
        
        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        # feature extraction layers
        x = input_x
        z = input_x
        for i in range(bi_layer_num-1):
            x = Bidirectional(pLSTM(unit))(x)
            if drop_rate: 
                x = SpatialDropout1D(rate=drop_rate)(x)
        # output layers
        x = Bidirectional(pLSTM(unit*2, return_sequences=False))(x)
        if drop_rate: 
            x = Dropout(rate=drop_rate)(x)
                
        for i in range(bi_layer_num-2):
            z = Bidirectional(pLSTM(unit))(z)
            if drop_rate: 
                z = SpatialDropout1D(rate=drop_rate)(z)
        z = Bidirectional(pLSTM(unit*2, return_sequences=False))(z)
        if drop_rate: 
            z = Dropout(rate=drop_rate)(z)
        z = pDense(100, activation='relu')(z)
        outputs_pho = pDense(19, activation='softmax', name='pho')(x)
        
        x = Concatenate()([x,z])
        x = pDense(512, activation='relu')(x)
        outputs_param = pDense(N_OUTPUTS, activation='linear', name='param')(x)
        
        model = keras.Model(inputs=input_x, outputs=[outputs_param, outputs_pho])
        # model.summary()
        return model
    return bilstm

In [27]:
model_pho = init_bilstm_pho()(24, 39)

In [28]:
model_pho.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 24, 39)]     0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 24, 256)      172032      input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 24, 256)      0           bidirectional_3[0][0]            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 24, 256)      172032      input_2[0][0]                    
____________________________________________________________________________________________

In [29]:
losses = {
    "param": "mse",
    "pho": tf.keras.losses.CategoricalCrossentropy(from_logits=True),
}
lossWeights = {"param": 1.0, "pho": 1.0}

In [30]:
metrics = {
    "param": [rmse, R2],
    "pho": "accuracy",
}

## Compile Model 

In [31]:
model_pho.compile(optimizer='adam',loss=losses, loss_weights=lossWeights, metrics=metrics)

## Train Model 

In [32]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 30

In [33]:
early = callbacks.EarlyStopping(monitor='param_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [34]:
history = model_pho.fit(X_train,{"param": y_train, "pho": z_train_oe},
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,{"param": y_val, "pho": z_val_oe}),
    callbacks=callback_list)

Train on 98746 samples, validate on 12094 samples
Epoch 1/30
98746/98746 - 54s - loss: 2.3965 - param_loss: 0.0713 - pho_loss: 2.3251 - param_rmse: 0.2419 - param_R2: 0.6899 - pho_accuracy: 0.7161 - val_loss: 2.2395 - val_param_loss: 0.0432 - val_pho_loss: 2.1963 - val_param_rmse: 0.1845 - val_param_R2: 0.8155 - val_pho_accuracy: 0.8358
Epoch 2/30
98746/98746 - 32s - loss: 2.2864 - param_loss: 0.0470 - pho_loss: 2.2394 - param_rmse: 0.1938 - param_R2: 0.7974 - pho_accuracy: 0.7928 - val_loss: 2.2200 - val_param_loss: 0.0391 - val_pho_loss: 2.1806 - val_param_rmse: 0.1743 - val_param_R2: 0.8325 - val_pho_accuracy: 0.8508
Epoch 3/30
98746/98746 - 32s - loss: 2.2617 - param_loss: 0.0411 - pho_loss: 2.2205 - param_rmse: 0.1814 - param_R2: 0.8224 - pho_accuracy: 0.8113 - val_loss: 2.2225 - val_param_loss: 0.0378 - val_pho_loss: 2.1845 - val_param_rmse: 0.1686 - val_param_R2: 0.8387 - val_pho_accuracy: 0.8463
Epoch 4/30
98746/98746 - 32s - loss: 2.2534 - param_loss: 0.0379 - pho_loss: 2.2155

In [35]:
model_pho.save('model_baseline_pho.hdf5')

## Evaluation

In [43]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
    print('Confusion Matrix')
    print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [39]:
y_pred, y_pho_pred = model_pho.predict(X_test)

In [40]:
err = compute_rmse(y_test,y_pred, axis=0)
print(err)

[0.16499175 0.13846306 0.15626041 0.20971853 0.16883098 0.22409996
 0.26204728 0.10337965 0.13968868 0.25097486 0.14635381 0.16778201
 0.18071192 0.27521861 0.18504289 0.15457323 0.15504013]


In [45]:
evaluate_model(y_pho_pred, z_test_oe)

Confusion Matrix
[[608   0   1  15   0   0   0   0   0   6  13   1   5  37   0   0   0   0
    0]
 [  0 668   0   1   0   0   0   0   2   0   0   1   0   0   0   0   0   2
   13]
 [  6   1 365   4   0   3   0   0   5  80   8  14  67  11   0   0   2   0
    0]
 [ 14   0   0 622   0   0   2   0   6   0   5   6   4   0   0   0   0   0
    0]
 [  1   0   0  12 595   0  31   1   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0 550   0   0   0   0  13   0  19  20   0   0   0   0
    0]
 [  0   0   0   1  12   0 614  36   0   0   1   0   0   1   0   0   0   0
    0]
 [  0   0   0   0   0   1  17 600   0   0   0   0   0   0   7   0   0   0
    0]
 [  0   1  18   4   0   0   0   0 598   0   0  29   0   0   0   0   0   0
    0]
 [  5   0  18   2   0   1   0   0   3 562   7  11  47   1   0   0   8   0
    0]
 [ 16   0   1   3   0  44   2   0   0   1 546   0  10  10   0   0   0   0
    0]
 [  5   1  16   0   0   0   0   0  54   5   0 537   0   0   0   0   5  24
    0]
 [  1   1  